# Risk Analysis for DonKa Detector

Initial tests to determine potential challenges for the DonKa Detector

## Assessments

 1. Options for audio streaming
 2. Optimizing for Digital Signal Processing (DSP)
    1. Might want to pivot away from Python. 
    2. First, see how intensive operations are involving NumPy
 3. High-accuracy detection options
    1. Create datasets of pencil taps and Tatacon usage
    2. Detection using statistics
       1. Quantile analysis of the DFT
       2. Mean/Median-based detectuib

### 1. Options for audio streaming

In [ ]:
import pyaudio
import wave
import math

FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 512
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "recordedFile.wav"
device_index = 2
audio = pyaudio.PyAudio()

In [ ]:
info = audio.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (audio.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", audio.get_device_info_by_host_api_device_index(0, i).get('name'))

In [ ]:
import time

device_index = 2
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,input_device_index = device_index,
                frames_per_buffer=CHUNK)

Recordframes = []
 
for i in range(0, math.ceil(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    Recordframes.append(data)

In [ ]:
stream.stop_stream()
stream.close()
audio.terminate()

In [ ]:
waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(Recordframes))
waveFile.close()

### 2. Options for DSP

Use the same setup as for assessment (1), but see how quickly we can perform operations using NumPy and librosa

#### Assessment of NumPy

In [ ]:
import pyaudio
import wave
import math

FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 4096
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "recordedFile.wav"
device_index = 2
audio = pyaudio.PyAudio()

info = audio.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (audio.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", audio.get_device_info_by_host_api_device_index(0, i).get('name'))

We can play around with different rates and chunk sizes

In [ ]:
import time
import numpy as np

device_index = 2
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,input_device_index = device_index,
                frames_per_buffer=CHUNK)

audio_arr = np.zeros((RATE, ))
np_time = 0
 
for i in range(0, math.ceil(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    start=time.time()
    # Shift Audio array
    audio_arr[:-CHUNK] = audio_arr[CHUNK:]
    audio_arr[-CHUNK:] = np.frombuffer(data, dtype=np.int16)
    np_time += time.time()-start

stream.stop_stream()
stream.close()
audio.terminate()

np_time/RECORD_SECONDS

In [ ]:
import matplotlib.pyplot as plt

plt.plot(audio_arr)

#### Assessment of Librosa

In [ ]:
import pyaudio
import wave
import math

FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 2048
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "recordedFile.wav"
device_index = 2
audio = pyaudio.PyAudio()

info = audio.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (audio.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", audio.get_device_info_by_host_api_device_index(0, i).get('name'))

In [ ]:
import time
import numpy as np
import librosa.onset

device_index = 2
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,input_device_index = device_index,
                frames_per_buffer=CHUNK)

audio_arr = np.zeros((RATE*RECORD_SECONDS + CHUNK, ))
onsets = np.array([audio_arr.shape[0]-1])
np_time,librosa_time = 0,0
 
for i in range(0, math.ceil(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    start=time.time()
    # Shift Audio array
    audio_arr[:-CHUNK] = audio_arr[CHUNK:]
    audio_arr[-CHUNK:] = np.frombuffer(data, dtype=np.int16)
    np_time += time.time()-start
    onsets -= CHUNK

    # Librosa onsets
    this_audio = audio_arr[onsets[-1]:]
    if np.max(np.abs(this_audio)) < 500: continue

    start = time.time()
    this_onsets = librosa.onset.onset_detect(y=this_audio, sr=RATE, units="samples")
    librosa_time += time.time() - start

    onsets = np.concatenate((onsets, this_onsets+onsets[-1]))


stream.stop_stream()
stream.close()
audio.terminate()

np_time/RECORD_SECONDS,librosa_time/RECORD_SECONDS

In [ ]:
import librosa.onset
import matplotlib.pyplot as plt

plt.plot([i/RATE for i in range(audio_arr.shape[0])],audio_arr)
plt.plot(onsets/RATE, [0]*len(onsets), 'o')
plt.show()